In [2]:
%matplotlib inline
%config InlineBackend.figure_format='svg'
#%config InlineBackend.figure_format='retina'

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.stats import LombScargle

from scipy import integrate
import astropy
from astropy.time import Time

import pyfits
%matplotlib inline
plt.style.use('seaborn-whitegrid')


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.stats import LombScargle
import matplotlib.ticker as ticker

In [ ]:
def get_tj(time,count):
    tj=[]
    for j in np.arange(time.size):
        if count[j]>0:
            tj.append(time[j]) 
    tj=np.array(tj)        
    return tj

def Integrate_phi(freqref,mudot):
    return 2*np.pi*(freqref*tj+0.5*mudot*tj**2) 
   
    

def get_phal(M,freqref,mudot):
    m=int(N/M)-1
    phi=Integrate_phi(freqref,mudot)
    phal=np.arange(M)
    for l in np.arange(M):
        phil=phi[l*m:(l+1)*m]
        phal[l]=np.arctan((np.sum(np.sin(phil))+np.sum(np.sin(2*phil))+np.sum(np.sin(3*phil)))/(np.sum(np.cos(phil))+np.sum(np.cos(2*phil))+np.sum(np.cos(3*phil))))        
    return phal  

def ZetaN(freqref,mudot):
    phi=Integrate_phi(freqref,mudot) 
    #n=3
    zeta3N=(np.sum(np.cos(phi)))**2+(np.sum(np.sin(phi)))**2+(np.sum(np.cos(2*phi)))**2+(np.sum(np.sin(2*phi)))**2+(np.sum(np.cos(3*phi)))**2+(np.sum(np.sin(3*phi)))**2
    #n=2
    zeta2N=(np.sum(np.cos(phi)))**2+(np.sum(np.sin(phi)))**2+(np.sum(np.cos(2*phi)))**2+(np.sum(np.sin(2*phi)))**2
    Zeta3N=zeta3N*2/N
    Zeta2N=zeta2N*2/N
    return Zeta3N,Zeta2N 

def chisq(M,freqref,mudot):
    phal=get_phal(M,freqref,mudot)
    #ZetaN2=ZetaN(freqref,mudot)
    chisq=(np.sum(phal**2)-np.sum(phal)**2/M)
    return chisq


In [ ]:
def get_time(gtistart,gtistop,time,count):
    new_time=[]
    new_count=[]
    for index,(time_ ,count_)in enumerate(zip(time,count)):
        if time_ >= gtistart and time_ <= gtistop:
            new_time.append(time_)
            new_count.append(count_)
    return new_time,new_count   


In [ ]:
lightcurve=pyfits.open('/Volumes/Brettlv_G_m/RX_J0806/4523/primary/src_lc_paper.fits')

In [ ]:
new_time,new_count=get_time(gtistart,gtistop,time,count)
print(new_time[0],new_time[-1])
print("gti_mjd_start",mjdzero+new_time[0]/86400)
print("gti_mjd_stop",mjdzero+new_time[-1]/86400)
plt.plot(new_time,new_count)
for index,start in enumerate(gtistart):
    plt.axvline(start,c='g')
    plt.axvline(gtistop[index],c='b')  

In [ ]:
new_time=new_time-new_time[0]
print(time.size)
tj=get_tj(new_time,new_count)
N=tj.size
print(tj.size)
x=np.linspace(0.003,0.004,1001)
#x=np.linspace(0.00311,0.003112,1001)
#x=np.linspace(0.003110303,0.003110305,2001)
#x=np.linspace(0.00310,0.00312,2001)
#x=np.linspace(0.00310967,0.00310969,2001)
#x=np.linspace(0.00311017,0.00311019,2001)

import os 
if os.path.exists("chandra%s.txt"%num):
    os.remove("chandra%s.txt"%num)
with open("chandra%s.txt"%num,"a") as f:
    for freq in x:       
        f.write(str(ZetaN(freq,0))+'\n')      

y=np.loadtxt("chandra%s.txt"%num,delimiter='\n')
#plt.xlim(0.003105,0.003120)
plt.figure(1,figsize=(4,3))
plt.xlabel('freq,mHz')
plt.ylabel('(Zn2)')
plt.gca().xaxis.set_major_formatter(ticker.FormatStrFormatter('%.4f'))
plt.plot(x*1000,y)